In [36]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [37]:
import datetime
from langchain.output_parsers import JsonOutputToolsParser, PydanticToolsParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

In [38]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert researcher.
            Current Time: {time}
            1. {first_instruction}
            2. Reflect and critique your answer. Be severe to maximize improvement.
            3. Recommend search queries to research information and improve your answer
            (NOTE: You must provide the search queries at all costs)"""
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
actor_prompt_template = actor_prompt_template.partial(time=lambda:datetime.datetime.now().isoformat())

In [39]:
from schemas import AnswerQuestion


llm = ChatOpenAI()
parser_json = JsonOutputToolsParser(return_id=True)
parser_pydantic = PydanticToolsParser(tools=[AnswerQuestion])

In [40]:
first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer"
)

In [41]:
first_responder = first_responder_prompt_template | llm.bind_tools(
    tools=[AnswerQuestion], tool_choice="AnswerQuestion"
)

In [42]:
human_message = HumanMessage(
    content="""Write about AI-Powered SOC / autonomous SOC problem domain
    List startups that do that and raised capital"""
)

In [43]:
chain = (
    first_responder_prompt_template 
    | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")
    | parser_pydantic
)

In [44]:
res = chain.invoke(input={"messages": [human_message]})

In [59]:
for inst in res[0]:
    print(inst[0], end=": ")
    print(inst[1])
    print()
    print()
    print()

answer: AI-Powered Security Operations Centers (SOC) or Autonomous SOC leverage artificial intelligence and machine learning to enhance threat detection, incident response, and overall cybersecurity posture. These solutions automate the monitoring of security events, analyze vast amounts of data to identify potential threats, and respond to incidents in real-time, reducing the workload on human analysts and improving response times. Autonomous SOCs are designed to proactively detect and mitigate cybersecurity threats before they cause significant damage.

Some startups in this domain include:
1. Darktrace: A leading AI cybersecurity company that has raised significant funding for its autonomous threat detection platform.
2. Vectra: Known for its AI-driven cybersecurity solutions for real-time threat detection and response.
3. Cynet: Offers a comprehensive autonomous breach protection platform powered by AI.
4. Exabeam: Provides AI-powered security intelligence and analytics for SOC tea